___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Arthur Alegro de Oliveira

#### Rafael dos Santos

#### José Antônio Bechara

___

# Classificação de Apps

___

## A. INTRODUÇÃO

Objetivo: Prever a nota de um app da PlayStore

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [1]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import pandas as pd
import json
import os

### BASE DE DADOS

In [2]:
# Reading Google Play Store data:
Google_data = pd.read_csv('googleplaystore.csv')

In [3]:
Google_data.head(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up


Ao analisar o *database* notamos que algumas colunas possuem dados numéricos em sua essência, mas por possuir outros caracteres são interpretados como texto. A seguir, serão destrinchados os problemas em cada umas dessas colunas. As funções que serão usadas já estão definidas abaixo:

In [4]:
def remove_last_char(word):
    new_word = word[:-1]
    return new_word

def remove_comma(word):
    new_word = "".join(word.split(','))
    return new_word

def remove_first_char(word):
    if '$' in word:
        new_word = word[1:]
        return new_word
    else:
        return word

Coluna `Size`:  
* possui o último caractere não numérico

In [5]:
Google_data['Size'] = Google_data['Size'].apply(remove_last_char)

 Coluna `Installs`:  
 * possui o último caractere não numérico
 * possui vírgulas como separador a cada $10^3$
 * possui dois tipos valores não itendificados (`Fre` e ` ` ) que serão descartados

In [6]:
Google_data['Installs'] = Google_data['Installs'].apply(remove_last_char)
Google_data['Installs'] = Google_data['Installs'].apply(remove_comma)
Google_data['Installs'] = Google_data['Installs'][Google_data['Installs'] != 'Fre']
Google_data['Installs'] = Google_data['Installs'][Google_data['Installs'] != '']

 Coluna `Price`: 
* possui o primeiro caractere $\$$ quando não é valor nulo (ou seja, quando o aplicativo é gratuito)
* possui um tipo de valor não identificado (`Everyone`) que será descartado

In [7]:
Google_data['Price'] = Google_data['Price'].apply(remove_first_char)
Google_data['Price'] = Google_data['Price'][Google_data['Price'] != 'Everyone']

Coluna `Reviews`:
* Possui um valor $3.0M$ (equivalente a $3000000$)

In [8]:
Google_data['Reviews'] = Google_data['Reviews'].mask(Google_data['Reviews'] == '3.0M', 3000000)

Com todas as filtragens e correções feitas, é possível modificar o tipo de dado para cada coluna de acordo com a necessidade. Logo, as colunas com valores qualitativos serão interpretadas como `category`, enquanto as colunas com valores quantitativos serão interpretadas como `float` ou `int`(dependendo se os valores originais possuem ou não casas decimais).

In [9]:
Google_data['Category'] = Google_data['Category'].astype('category')
Google_data['Rating'] = Google_data['Rating'].astype('float64')
Google_data['Reviews'] = Google_data['Reviews'].astype('float64')
Google_data['Installs'] = Google_data['Installs'].astype('float64')
Google_data['Price'] = Google_data['Price'].astype('float64')
Google_data['Content Rating'] = Google_data['Content Rating'].astype('category')
Google_data['Genres'] = Google_data['Genres'].astype('category')

In [10]:
usefull_columns = ["App", "Category", "Rating", "Reviews", "Installs", "Price", "Content Rating", "Genres"]
Google_data = Google_data[usefull_columns]

App --> str, ex:'Photo Editor & Candy Camera & Grid & ScrapBook' <br>
Category --> str, ex: 'ART_AND_DESIGN' <br>
Rating --> float, ex: 4.0999999 <br>
Reviews --> str, ex: 159 <br>
Installs --> str, ex: 10,000+ <br>
Price --> str, ex: '0' <br>
Content Rating --> str, ex: Everyone <br>
Genres --> str, ex: 'Art & Design' <br>

In [11]:
def dummify(data, column_name):
    df = data.copy()
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

In [12]:
Google_data = dummify(Google_data, "Category")
Google_data = dummify(Google_data, "Content Rating")
Google_data = dummify(Google_data, "Genres")

### ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

In [61]:
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

def variablesLessAlpha(df, listX, y, alpha=0.05):
    '''
    Exclui todas as variaveis em que o P>|t| for maior que o alpha e retorna uma regressão linear
    
    df --> DataFrame
    listX --> Lista de variaveis independentes
    y --> Variavel dependente/a ser analizada 
    alpha --> nivel de significancia, 5% como padrão'''
    
    y = df[[y]]
    xs = df[listX]
    results = regress(xs, y)
    
    actual_max = max(results.pvalues)
    while actual_max > alpha:
        actual_max = max(results.pvalues)
        j = 0
        if actual_max > alpha:
            for e in list(results.pvalues):
                j += 1
                if e == actual_max:
                    break
            listX.remove(listX[j-2])
            xs = df[listX]
            results = regress(xs, y)
            
    return results

In [62]:
PlayStore_X = list(Google_data.columns)
Google_data = Google_data.dropna()
PlayStore_X.remove("App")
PlayStore_X.remove("Rating")
PlayStore_Y = 'Rating'

resultsGoogle = variablesLessAlpha(Google_data, PlayStore_X, PlayStore_Y)
resultsGoogle.summary()


/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Rating   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     12.14
Date:                Tue, 21 May 2019   Prob (F-statistic):           9.25e-67
Time:                        14:30:45   Log-Likelihood:                -6869.5
No. Observations:                9366   AIC:                         1.381e+04
Df Residuals:                    9330   BIC:                         1.407e+04
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                            4.1358      0.017    245.610      0.000       4.103       4.169
Reviews                       9.492e-09   1.71e-09      5.564      0.000    6.15e-09    1.28e-08
Category_ART_AND_DESIGN          0.2553      0.065      3.943      0.000       0.128       0.382
Category_BEAUTY                  0.0879      0.039      2.241      0.025       0.011       0.165
Category_BOOKS_AND_REFERENCE     0.1196      0.019      6.144      0.000       0.081       0.158
Category_DATING                 -0.0818      0.020     -4.131      0.000      -0.121      -0.043
Category_EDUCATION               0.2837      0.042      6.829      0.000       0.202       0.365
Category_ENTERTAINMENT           0.1796      0.051      3.537      0.000       0.080       0.279
Category_EVENTS                  0.1641      0.038      4.329      0.000       0.090       0.238
Category_FAMILY                  0.1719      0.019      9.240      0.000       0.135       0.208
Category_GAME                    0.3743      0.040      9.416      0.000       0.296       0.452
Category_HEALTH_AND_FITNESS      0.0861      0.015      5.615      0.000       0.056       0.116
Category_MEDICAL                 0.0436      0.014      3.063      0.002       0.016       0.071
Category_PERSONALIZATION         0.1142      0.015      7.643      0.000       0.085       0.144
Category_PHOTOGRAPHY             0.0418      0.015      2.810      0.005       0.013       0.071
Category_PRODUCTIVITY            0.0539      0.014      3.788      0.000       0.026       0.082
Category_SHOPPING                0.0736      0.017      4.331      0.000       0.040       0.107
Category_SOCIAL                  0.0543      0.017      3.197      0.001       0.021       0.088
Category_SPORTS                  0.1163      0.030      3.919      0.000       0.058       0.174
Category_WEATHER                 0.0700      0.029      2.375      0.018       0.012       0.128
Content Rating_Everyone         -0.0357      0.016     -2.301      0.021      -0.066      -0.005
Genres_Action                   -0.2246      0.048     -4.709      0.000      -0.318      -0.131
Genres_Adventure                -0.3200      0.071     -4.509      0.000      -0.459      -0.181
Genres_Arcade                   -0.1973      0.052     -3.785      0.000      -0.299      -0.095
Genres_Beauty                    0.0879      0.039      2.241      0.025       0.011       0.165
Genres_Board                    -0.1880      0.088     -2.140      0.032      -0.360      -0.016
Genres_Books & Reference         0.1196      0.019      6.144      0.000       0.081       0.158
Genres_Card                     -0.4067      0.085     -4.797      0.000      -0.573      -0.240
Genres_Casino                   -0.2230      0.092     -2.415      0.016    

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS